# Clustering Internals: Architecture and Data Structures

A deep dive into how time series clustering works under the hood.

This notebook covers:

- **Module overview**: The `flixopt.aggregation` module and its classes
- **Data flow**: From `cluster()` through optimization to `expand_solution()`
- **Core classes**: `ClusterStructure`, `ClusterResult`, `ClusterInfo`
- **Cluster weights**: How operational costs are scaled correctly
- **Storage linking**: Inter-cluster constraints for realistic storage behavior
- **Multi-dimensional support**: Handling periods and scenarios

!!! note "Prerequisites"
    This notebook assumes familiarity with [08c-clustering](08c-clustering.ipynb).

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import flixopt as fx

fx.CONFIG.notebook()

In [ ]:
# Load the district heating system
data_file = Path('data/district_heating_system.nc4')
if not data_file.exists():
    from data.generate_example_systems import create_district_heating_system

    fs = create_district_heating_system()
    fs.to_netcdf(data_file)

flow_system = fx.FlowSystem.from_netcdf(data_file)
print(f'Loaded: {len(flow_system.timesteps)} timesteps ({len(flow_system.timesteps) / 96:.0f} days)')

## 1. Module Architecture Overview

The clustering functionality lives in `flixopt.aggregation` with this structure:

```
flixopt.aggregation/
├── base.py           # Core dataclasses: ClusterStructure, ClusterResult, ClusterInfo
├── storage_linking.py # InterClusterLinking for storage constraints
└── __init__.py       # Public exports
```

### Key Classes

| Class | Purpose |
|-------|--------|
| `ClusterStructure` | Hierarchical structure: which original periods map to which clusters |
| `ClusterResult` | Universal container: timestep mapping, weights, aggregated data |
| `ClusterInfo` | Stored on FlowSystem after clustering; enables `expand_solution()` |
| `InterClusterLinking` | Adds storage SOC constraints across the original time horizon |

In [ ]:
# Import the aggregation module to explore its contents
from flixopt import aggregation

print('Available in flixopt.aggregation:')
print([name for name in dir(aggregation) if not name.startswith('_')])

## 2. Data Flow: From `cluster()` to `expand_solution()`

```
┌─────────────────────────────────────────────────────────────────┐
│  flow_system.transform.cluster(n_clusters=8, ...)               │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│  1. Extract time series data from FlowSystem                    │
│  2. Call tsam for clustering                                    │
│  3. Build ClusterStructure (cluster_order, occurrences)         │
│  4. Build ClusterResult (timestep_mapping, weights)             │
│  5. Create reduced FlowSystem with representative timesteps     │
│  6. Store ClusterInfo on reduced_fs._cluster_info               │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│  reduced_fs.optimize(solver)                                    │
│  └─ InterClusterLinking adds storage constraints if enabled     │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│  reduced_fs.transform.expand_solution()                         │
│  └─ Uses stored timestep_mapping to expand back to full time    │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Create a clustered system
fs_clustered = flow_system.transform.cluster(
    n_clusters=8,
    cluster_duration='1D',
    time_series_for_high_peaks=['HeatDemand(Q_th)|fixed_relative_profile'],
)

print(f'Original timesteps:  {len(flow_system.timesteps)}')
print(f'Clustered timesteps: {len(fs_clustered.timesteps)}')
print(f'Reduction: {len(flow_system.timesteps) / len(fs_clustered.timesteps):.1f}x')

## 3. The `ClusterInfo` Structure

After clustering, metadata is stored in `fs._cluster_info`. This enables:
- Expanding solutions back to full resolution
- Understanding which original days map to which clusters
- Correct weighting in the objective function

In [ ]:
info = fs_clustered._cluster_info

print('ClusterInfo attributes:')
print(f'  backend_name:                 {info.backend_name}')
print(f'  storage_inter_cluster_linking: {info.storage_inter_cluster_linking}')
print(f'  storage_cyclic:               {info.storage_cyclic}')
print(f'  original_flow_system:         {type(info.original_flow_system).__name__}')
print(f'  result:                       {type(info.result).__name__}')

## 4. The `ClusterStructure`: Hierarchical Mapping

The `ClusterStructure` captures which original periods (days) belong to which clusters:

- **`cluster_order`**: Array mapping each original period index to its cluster ID
- **`cluster_occurrences`**: How many original periods each cluster represents
- **`n_clusters`**: Number of representative clusters
- **`timesteps_per_cluster`**: Timesteps in each cluster (e.g., 96 for daily with 15-min resolution)

In [ ]:
cs = info.result.cluster_structure

print('ClusterStructure:')
print(f'  n_clusters:           {cs.n_clusters}')
print(f'  timesteps_per_cluster: {cs.timesteps_per_cluster}')
print(f'  n_original_periods:   {cs.n_original_periods}')
print(f'  cluster_order dims:   {cs.cluster_order.dims}')
print(f'  cluster_order shape:  {cs.cluster_order.shape}')

In [ ]:
# cluster_order shows which cluster each original day belongs to
cluster_order = cs.cluster_order.values

print('Cluster assignments (first 14 days):')
for day in range(min(14, len(cluster_order))):
    print(f'  Day {day + 1:2d} → Cluster {cluster_order[day]}')

In [ ]:
# cluster_occurrences shows how many original days each cluster represents
print('Cluster occurrences (days per cluster):')
for cluster_id in range(cs.n_clusters):
    count = int(cs.cluster_occurrences.sel(cluster=cluster_id).values)
    print(f'  Cluster {cluster_id}: {count} day(s)')

print(f'\nTotal: {int(cs.cluster_occurrences.sum().values)} days')

In [ ]:
# Visualize cluster assignment
days_df = pd.DataFrame(
    {
        'Day': range(1, cs.n_original_periods + 1),
        'Cluster': cluster_order,
    }
)

fig = px.bar(
    days_df,
    x='Day',
    y=[1] * len(days_df),
    color='Cluster',
    color_continuous_scale='Viridis',
    title='Cluster Assignment by Day',
)
fig.update_layout(height=250, yaxis_visible=False, coloraxis_colorbar_title='Cluster')
fig.show()

## 5. The `ClusterResult`: Timestep Mapping and Weights

The `ClusterResult` contains:

- **`timestep_mapping`**: Maps each original timestep to its representative timestep index
- **`representative_weights`**: Weight for each representative timestep (used as `cluster_weight`)
- **`cluster_structure`**: Reference to the hierarchical structure
- **`original_data`**: The time series data used for clustering

In [ ]:
result = info.result

print('ClusterResult:')
print(f'  n_representatives:        {result.n_representatives}')
print(f'  timestep_mapping dims:    {result.timestep_mapping.dims}')
print(f'  timestep_mapping shape:   {result.timestep_mapping.shape}')
print(f'  representative_weights:   {result.representative_weights.shape}')

In [ ]:
# The timestep_mapping shows which representative timestep each original timestep maps to
mapping = result.timestep_mapping.values

print('Timestep mapping (first 10 original timesteps):')
for t in range(10):
    print(f'  Original t={t} → Representative t={mapping[t]}')

print(f'\n... (total {len(mapping)} mappings)')

## 6. Cluster Weights: Scaling Operational Costs

When optimizing over typical periods, operational costs must be **scaled** to represent the full time horizon.

### The Weight Formula

$$\text{Objective} = \sum_{t \in \text{typical}} w_t \cdot c_t$$

Where:
- $w_t$ = cluster weight for timestep $t$ (number of original days this cluster represents)
- $c_t$ = operational cost at timestep $t$

### Weight Conservation

$$\sum_{t \in \text{typical}} w_t = |\text{original timesteps}|$$

In [ ]:
# The cluster_weight is stored on the FlowSystem
print('cluster_weight on FlowSystem:')
print(f'  Shape: {fs_clustered.cluster_weight.shape}')
print(f'  Sum:   {fs_clustered.cluster_weight.sum().item():.0f}')
print(f'  Expected (original timesteps): {len(flow_system.timesteps)}')

In [ ]:
# Visualize weights across timesteps
weights = fs_clustered.cluster_weight.values

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=list(range(len(weights))),
        y=weights,
        mode='lines',
        name='Cluster Weight',
    )
)

# Add vertical lines at cluster boundaries
for i in range(1, cs.n_clusters):
    fig.add_vline(x=i * cs.timesteps_per_cluster, line_dash='dash', line_color='gray', opacity=0.5)

fig.update_layout(
    height=300,
    title='Cluster Weight per Timestep',
    xaxis_title='Timestep Index',
    yaxis_title='Weight',
)
fig.show()

## 7. Storage Inter-Cluster Linking

Storage behavior requires special handling in clustering. The `InterClusterLinking` class:

1. Creates **SOC_boundary** variables for each original period boundary
2. Computes **delta_SOC** for each representative period (change in SOC)
3. Links them: `SOC_boundary[d+1] = SOC_boundary[d] + delta_SOC[cluster_order[d]]`
4. Optionally enforces cyclic constraint: `SOC_boundary[0] = SOC_boundary[end]`

This tracks storage state across the **full original time horizon** while only solving for representative periods.

In [ ]:
print('Storage settings in ClusterInfo:')
print(f'  storage_inter_cluster_linking: {info.storage_inter_cluster_linking}')
print(f'  storage_cyclic: {info.storage_cyclic}')

In [ ]:
# Optimize and examine storage behavior
solver = fx.solvers.HighsSolver(mip_gap=0.01, log_to_console=False)
fs_clustered.optimize(solver)

# Check storage charge state
if 'Storage|charge_state' in fs_clustered.solution:
    charge_state = fs_clustered.solution['Storage|charge_state']
    print(f'Charge state shape: {charge_state.shape}')
    print(f'Initial charge: {charge_state.values[0]:.1f} MWh')
    print(f'Final charge: {charge_state.values[-1]:.1f} MWh')
else:
    print('No storage in this system')

## 8. Multi-Dimensional Support (Periods/Scenarios)

When a FlowSystem has multiple **periods** (e.g., investment years) or **scenarios**, each (period, scenario) combination may have **different cluster assignments**.

The data structures support this with multi-dimensional arrays:

```python
# Simple case (no periods/scenarios)
cluster_order.dims = ['original_period']
timestep_mapping.dims = ['original_time']

# Multi-scenario case
cluster_order.dims = ['original_period', 'scenario']
timestep_mapping.dims = ['original_time', 'scenario']
```

Helper methods extract per-slice data:
```python
cluster_structure.get_cluster_order_for_slice(period='2025', scenario='high')
cluster_result.get_timestep_mapping_for_slice(scenario='base')
```

In [ ]:
# Check if our system has multi-dimensional clustering
print('Multi-dimensional check:')
print(f'  cluster_order dims: {cs.cluster_order.dims}')
print(f'  has_multi_dims: {cs.has_multi_dims}')

# Get cluster order (works for both simple and multi-dim cases)
cluster_order = cs.get_cluster_order_for_slice()
print(f'  cluster_order shape: {cluster_order.shape}')

## 9. Expanding Solutions Back to Full Resolution

After optimization, `expand_solution()` uses the stored `timestep_mapping` to map typical period results back to the original time horizon:

In [ ]:
# Expand the solution
fs_expanded = fs_clustered.transform.expand_solution()

print(f'Clustered timesteps: {len(fs_clustered.timesteps)}')
print(f'Expanded timesteps:  {len(fs_expanded.timesteps)}')
print(f'Original timesteps:  {len(flow_system.timesteps)}')

In [ ]:
# The expanded solution has full time resolution
if 'Boiler(Q_th)|flow_rate' in fs_expanded.solution:
    flow_clustered = fs_clustered.solution['Boiler(Q_th)|flow_rate']
    flow_expanded = fs_expanded.solution['Boiler(Q_th)|flow_rate']

    print(f'Clustered flow shape: {flow_clustered.shape}')
    print(f'Expanded flow shape:  {flow_expanded.shape}')

## 10. Visualizing Typical vs Original Data

In [ ]:
# Get heat demand from original and clustered systems
original_demand = flow_system.components['HeatDemand'].inputs[0].fixed_relative_profile.values
clustered_demand = fs_clustered.components['HeatDemand'].inputs[0].fixed_relative_profile.values

# Reshape into days
timesteps_per_day = cs.timesteps_per_cluster
n_days = len(original_demand) // timesteps_per_day
original_by_day = original_demand[: n_days * timesteps_per_day].reshape(n_days, timesteps_per_day)
clustered_by_day = clustered_demand.reshape(cs.n_clusters, timesteps_per_day)

# Create subplots
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=[f'Original: All {n_days} Days', f'Clustered: {cs.n_clusters} Typical Days'],
    vertical_spacing=0.15,
)

hours = np.arange(timesteps_per_day) / 4  # Convert to hours

# Plot all original days (faded)
for day in range(n_days):
    fig.add_trace(
        go.Scatter(
            x=hours, y=original_by_day[day], mode='lines', line=dict(width=0.5, color='lightblue'), showlegend=False
        ),
        row=1,
        col=1,
    )

# Plot typical days
colors = px.colors.qualitative.Set1
for cluster_id in range(cs.n_clusters):
    weight = int(cs.cluster_occurrences.sel(cluster=cluster_id).values)
    fig.add_trace(
        go.Scatter(
            x=hours,
            y=clustered_by_day[cluster_id],
            mode='lines',
            name=f'Cluster {cluster_id} (x{weight})',
            line=dict(width=2, color=colors[cluster_id % len(colors)]),
        ),
        row=2,
        col=1,
    )

fig.update_layout(height=600, title='Heat Demand: Original vs Typical Days')
fig.update_xaxes(title_text='Hour of Day', row=2, col=1)
fig.update_yaxes(title_text='MW')
fig.show()

## Summary

### Module Structure

```
flixopt.aggregation
├── ClusterStructure   # Hierarchical: cluster_order, occurrences
├── ClusterResult      # Container: timestep_mapping, weights
├── ClusterInfo        # Stored on FlowSystem._cluster_info
└── InterClusterLinking # Storage SOC constraints
```

### Data Flow

1. `cluster()` → Creates reduced FlowSystem + stores `ClusterInfo`
2. `optimize()` → `InterClusterLinking` adds storage constraints
3. `expand_solution()` → Uses `timestep_mapping` to restore full time

### Key Formulas

| Formula | Description |
|---------|-------------|
| $\sum w_t \cdot c_t$ | Weighted objective function |
| $\sum w_t = N_{\text{original}}$ | Weight conservation |
| $SOC_{d+1} = SOC_d + \Delta SOC_{c[d]}$ | Inter-cluster storage linking |